In [1]:
import sys
sys.path.append("/home/jovyan/work/Architectures_for_Big_Data/")
import pyspark
sc = pyspark.SparkContext("local[3]")

In [2]:
from dataGenerator.rowGenerator import *
from dataGenerator.datasetGenerator import *
from datetime import datetime

dataset = datasetGenerator()
dataset.addGenerator(idGenerator(),prefix="book",min=1,max=100,keyName="bookId")
dataset.addGenerator(idGenerator(),prefix="token",min=1,max=5000,keyName="tokenId")


rdd = sc.parallelize( dataset.generateDataset(250000)).persist()
rdd.first()

{'bookId': 'book_0060', 'tokenId': 'token_01376'}

In [5]:
# tf
tf = rdd.map(lambda x: ((x.get('bookId'), x.get('tokenId')), 1)).reduceByKey(lambda a,b: a+b)
tf.take(5)

[(('book_0009', 'token_01605'), 2),
 (('book_0045', 'token_03798'), 3),
 (('book_0083', 'token_03345'), 1),
 (('book_0002', 'token_02261'), 1),
 (('book_0066', 'token_02110'), 3)]

In [8]:
# df
df = tf.map(lambda x: (x[0][1], 1)).reduceByKey(lambda a,b: a+b)
df.take(5)

[('token_03345', 39),
 ('token_01486', 37),
 ('token_03589', 37),
 ('token_01101', 37),
 ('token_04054', 48)]

In [16]:
# idf
import numpy as np
totBooks = rdd.map(lambda x: x.get('bookId')).distinct().count()
idf = df.map(lambda x: (x[0], np.log(totBooks/x[1])))
idf.take(5)

[('token_03345', 0.9416085398584451),
 ('token_01486', 0.9942522733438669),
 ('token_03589', 0.9942522733438669),
 ('token_01101', 0.9942522733438669),
 ('token_04054', 0.7339691750802005)]

In [22]:
tfidf = tf.map(lambda x: (x[0][1], x)).join(idf).map(lambda x: (x[0], x[1][0][0][0], x[1][0][1], x[1][1], x[1][0][1]*x[1][1]))
tfidf.top(5, lambda x: 1/x[4])

[('token_04279', 'book_0086', 1, 0.5447271754416719, 0.5447271754416719),
 ('token_04279', 'book_0075', 1, 0.5447271754416719, 0.5447271754416719),
 ('token_04279', 'book_0002', 1, 0.5447271754416719, 0.5447271754416719),
 ('token_04279', 'book_0030', 1, 0.5447271754416719, 0.5447271754416719),
 ('token_04279', 'book_0062', 1, 0.5447271754416719, 0.5447271754416719)]